In [ ]:
#hacer todo esto de las provincias españolas
#poblacion, salario minimo, porcentaje de jovenes, si tienen ya un parque o no, numero de visitantes (extranjeros o locales)

In [54]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


In [55]:
poblacion_españa = pd.read_csv('C:/Users/noeli/Downloads/2852.csv', sep=';') 

print(poblacion_españa.columns)

print(poblacion_españa['Periodo'].dtype)

#Quedarme solo con 2021
población_españa = poblacion_españa[poblacion_españa['Periodo'] == 2021]
poblacion_españa['Periodo'] = pd.to_numeric(poblacion_españa['Periodo'], errors='coerce')
poblacion_españa_filtrada = poblacion_españa[poblacion_españa['Periodo'] == 2021]
print(poblacion_españa_filtrada)


Index(['Provincias', 'Sexo', 'Periodo', 'Total'], dtype='object')
int64
       Provincias     Sexo  Periodo       Total
0           Total    Total     2021  47.385.107
26          Total  Hombres     2021  23.222.953
52          Total  Mujeres     2021  24.162.154
78    02 Albacete    Total     2021     386.464
104   02 Albacete  Hombres     2021     193.205
...           ...      ...      ...         ...
4004     51 Ceuta  Hombres     2021      42.208
4030     51 Ceuta  Mujeres     2021      41.309
4056   52 Melilla    Total     2021      86.261
4082   52 Melilla  Hombres     2021      43.603
4108   52 Melilla  Mujeres     2021      42.658

[159 rows x 4 columns]


In [56]:
definitivo = poblacion_españa_filtrada.pivot(index='Provincias', columns='Sexo', values='Total').reset_index()


In [57]:
definitivo

Sexo,Provincias,Hombres,Mujeres,Total
0,01 Araba/Álava,164.252,169.374,333.626
1,02 Albacete,193.205,193.259,386.464
2,03 Alicante/Alacant,932.018,949.744,1.881.762
3,04 Almería,373.136,358.656,731.792
4,05 Ávila,79.562,78.859,158.421
5,06 Badajoz,331.025,338.918,669.943
6,"07 Balears, Illes",584.853,588.155,1.173.008
7,08 Barcelona,2.791.250,2.923.480,5.714.730
8,09 Burgos,177.920,178.135,356.055
9,10 Cáceres,192.951,196.607,389.558


In [58]:
import re
definitivo["Provincias"] = definitivo["Provincias"].str.replace(r"^\d+\s*", "", regex=True)


In [122]:
definitivo

Sexo,Provincias,Hombres,Mujeres,Total
0,Araba/Álava,164.252,169.374,333.626
1,Albacete,193.205,193.259,386.464
2,Alicante/Alacant,932.018,949.744,1.881.762
3,Almería,373.136,358.656,731.792
4,Ávila,79.562,78.859,158.421
5,Badajoz,331.025,338.918,669.943
6,"Balears, Illes",584.853,588.155,1.173.008
7,Barcelona,2.791.250,2.923.480,5.714.730
8,Burgos,177.920,178.135,356.055
9,Cáceres,192.951,196.607,389.558


In [59]:
pip install dateparser

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd

# Ruta del archivo Excel
archivo = "C:/Users/noeli/Downloads/9.-Estadisticas_Septiembre_2025.xlsx"

hoja = 0  

# Leer solo el rango B6:D56
df = pd.read_excel(
    archivo,
    sheet_name=hoja,
    usecols="B:D",
    skiprows=5, 
    nrows=51      
)


In [123]:
import pandas as pd
aeropuertos_comunidades = pd.read_excel("C:/Users/noeli/Downloads/aeropuertos_comerciales_españa.xlsx")

In [124]:
aeropuertos1 = aeropuertos_comunidades.drop(columns=["Código IATA", "Ciudad principal", "Comunidad Autónoma"])
aeropuertos2 = aeropuertos1.rename(columns={"Provincia": "Provincias"})

In [18]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [125]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


url = "https://sede.agenciatributaria.gob.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/mercado/2022/jrubikf1b31c8858155bdb0bed1e4e7d45a9e38faf2d78d.html"


resp = requests.get(url)
resp.encoding = resp.apparent_encoding 
html = resp.text


soup = BeautifulSoup(html, "html.parser")


table = soup.find("table")


renta = pd.read_html(str(table))[0]




C:\Users\noeli\AppData\Local\Temp\ipykernel_33468\1770819690.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  renta = pd.read_html(str(table))[0]


In [126]:
renta_actualizada = renta.rename(columns={"Unnamed: 0": "Provincias"})

In [127]:
renta_actualizada['Provincias'] = renta_actualizada['Provincias'].replace({
    "Asturias, Principado de": "Asturias",
    "Madrid, Comunidad de": "Madrid",
    "S.C.Tenerife": "Tenerife",
    "Murcia, Región de": "Murcia", "Alicante": "Alicante/Alacant", "Castellón": "Castellón/Castelló", "S.C.Tenerife": "Santa Cruz de Tenerife", "Valencia": "Valencia/València"
})



In [128]:
valores_a_borrar = ["Total","Andalucía", "Aragón", "Canarias", "Castilla y León", "Castilla - La Mancha", "Cataluña", "Comunitat Valenciana", "Extremadura","Galicia"]

renta_actualizada3 = renta_actualizada[~renta_actualizada['Provincias'].isin(valores_a_borrar)]


In [129]:
renta_actualizada3.drop(columns=["Asalariados", "Percepciones por persona", "Salarios"])

,Provincias,Salario Medio Anual
2,Almería,16.962
3,Cádiz,18.988
4,Córdoba,17.580
5,Granada,18.658
6,Huelva,16.190
7,Jaén,16.012
8,Málaga,19.443
9,Sevilla,19.851
11,Huesca,21.026
12,Teruel,20.777


In [130]:
renta_actualizada2 = renta_actualizada3.sort_values(by="Provincias").reset_index(drop=True)


In [131]:
provincias_salarios = pd.merge(definitivo, renta_actualizada2, on="Provincias", how="outer")


In [132]:
provincias_salarios2 = provincias_salarios.drop(columns=["Asalariados", "Percepciones por persona", "Salarios"])


In [133]:
provincias3 = provincias_salarios2.rename(columns={"Total": "Personas Totales"})

In [134]:
import pandas as pd


In [135]:
aeropuerto = pd.read_excel("C:/Users/noeli/Downloads/aeropuertos_comerciales_españa.xlsx")

In [136]:
aeropuertos4 = aeropuerto.drop(columns=["Código IATA", "Ciudad principal", "Comunidad Autónoma"])

In [137]:
aeropuertos2 = aeropuertos4.rename(columns={"Provincia": "Provincias"})

In [142]:
local = pd.merge(provincias3, aeropuertos2, on= "Provincias", how= "outer" )

In [143]:
local

,Provincias,Hombres,Mujeres,Personas Totales,Salario Medio Anual,Aeropuerto
0,A Coruña,NaN,NaN,NaN,23.456,Aeropuerto de A Coruña
1,Albacete,193.205,193.259,386.464,19.680,NaN
2,Alicante,NaN,NaN,NaN,NaN,Aeropuerto de Alicante-Elche Miguel Hernández
3,Alicante/Alacant,932.018,949.744,1.881.762,19.112,NaN
4,Almería,373.136,358.656,731.792,16.962,Aeropuerto de Almería
...,...,...,...,...,...,...
65,Valencia/València,1.267.961,1.321.351,2.589.312,22.248,NaN
66,Valladolid,252.843,266.518,519.361,23.407,Aeropuerto de Valladolid
67,Zamora,83.551,85.174,168.725,19.254,NaN
68,Zaragoza,473.354,494.098,967.452,23.346,Aeropuerto de Zaragoza


In [220]:
import pandas as pd

In [150]:
import pandas as pd

edad4 = pd.read_csv("C:/Users/noeli/Downloads/3199.csv", sep=';', on_bad_lines='skip')



In [176]:
edad_2022 = edad4[edad4['Periodo'] == 2022]
edad_= edad_2022[edad_2022['Sexo'] == "Ambos sexos"]
edad_ = edad_[edad_['Provincias'] != "Total Nacional"]
import re
edad_["Provincias"] = edad_["Provincias"].str.replace(r"^\d+\s*", "", regex=True)
edad_.rename(columns={"Total": "Edad Media de la Provincia"}, inplace=True)
edad_['Provincias'] = edad_['Provincias'].replace({
    "Coruña, A": "A Coruña"
})


edad_1 = edad_.drop(columns=["Sexo", "Periodo"])


In [177]:
edad_1

,Provincias,Edad Media de la Provincia
152,Albacete,"44,14"
302,Alicante/Alacant,"44,28"
452,Almería,"40,78"
602,Araba/Álava,"45,05"
752,Asturias,"49,16"
902,Ávila,"48,24"
1052,Badajoz,"44,24"
1202,"Balears, Illes","42,08"
1352,Barcelona,"43,38"
1502,Bizkaia,"46,58"


In [178]:
final = pd.merge(local, edad_1, on= "Provincias", how= "outer" )

In [181]:
# Suponiendo que tienes esta lista de columnas
columnas = final.columns.tolist()

columnas.remove("Edad Media de la Provincia")

# Insertas al principio
columnas.insert(4, "Edad Media de la Provincia")

# Reordenas el DataFrame
final = final[columnas]


In [182]:
final

,Provincias,Hombres,Mujeres,Personas Totales,Edad Media de la Provincia,Salario Medio Anual,Aeropuerto
0,A Coruña,NaN,NaN,NaN,"47,71",23.456,Aeropuerto de A Coruña
1,Albacete,193.205,193.259,386.464,"44,14",19.680,NaN
2,Alicante,NaN,NaN,NaN,NaN,NaN,Aeropuerto de Alicante-Elche Miguel Hernández
3,Alicante/Alacant,932.018,949.744,1.881.762,"44,28",19.112,NaN
4,Almería,373.136,358.656,731.792,"40,78",16.962,Aeropuerto de Almería
...,...,...,...,...,...,...,...
65,Valencia/València,1.267.961,1.321.351,2.589.312,"43,87",22.248,NaN
66,Valladolid,252.843,266.518,519.361,"46,71",23.407,Aeropuerto de Valladolid
67,Zamora,83.551,85.174,168.725,"51,48",19.254,NaN
68,Zaragoza,473.354,494.098,967.452,"44,97",23.346,Aeropuerto de Zaragoza
